In [1]:
!pip install apyori
!pip install mlxtend


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from apyori import apriori
import utils
%matplotlib inline

In [3]:
dataset = pd.read_csv('in-vehicle-coupon-recommendation.csv')

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12684 entries, 0 to 12683
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   destination           12684 non-null  object
 1   passanger             12684 non-null  object
 2   weather               12684 non-null  object
 3   temperature           12684 non-null  int64 
 4   time                  12684 non-null  object
 5   coupon                12684 non-null  object
 6   expiration            12684 non-null  object
 7   gender                12684 non-null  object
 8   age                   12684 non-null  object
 9   maritalStatus         12684 non-null  object
 10  has_children          12684 non-null  int64 
 11  education             12684 non-null  object
 12  occupation            12684 non-null  object
 13  income                12684 non-null  object
 14  car                   108 non-null    object
 15  Bar                   12577 non-null

In [5]:
dataset.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0


In [6]:
dataset = dataset.drop(['temperature', 'age','education', 'occupation'], axis=1)


In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12684 entries, 0 to 12683
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   destination           12684 non-null  object
 1   passanger             12684 non-null  object
 2   weather               12684 non-null  object
 3   time                  12684 non-null  object
 4   coupon                12684 non-null  object
 5   expiration            12684 non-null  object
 6   gender                12684 non-null  object
 7   maritalStatus         12684 non-null  object
 8   has_children          12684 non-null  int64 
 9   income                12684 non-null  object
 10  car                   108 non-null    object
 11  Bar                   12577 non-null  object
 12  CoffeeHouse           12467 non-null  object
 13  CarryAway             12533 non-null  object
 14  RestaurantLessThan20  12554 non-null  object
 15  Restaurant20To50      12495 non-null

In [8]:
dataset.head()

,destination,passanger,weather,time,coupon,expiration,gender,maritalStatus,has_children,income,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,2PM,Restaurant(<20),1d,Female,Unmarried partner,1,$37500 - $49999,...,never,NaN,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,10AM,Coffee House,2h,Female,Unmarried partner,1,$37500 - $49999,...,never,NaN,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,10AM,Carry out & Take away,2h,Female,Unmarried partner,1,$37500 - $49999,...,never,NaN,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,2PM,Coffee House,2h,Female,Unmarried partner,1,$37500 - $49999,...,never,NaN,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,2PM,Coffee House,1d,Female,Unmarried partner,1,$37500 - $49999,...,never,NaN,4~8,1~3,1,1,0,0,1,0


In [9]:
dataset.isnull().sum()

destination                 0
passanger                   0
weather                     0
time                        0
coupon                      0
expiration                  0
gender                      0
maritalStatus               0
has_children                0
income                      0
car                     12576
Bar                       107
CoffeeHouse               217
CarryAway                 151
RestaurantLessThan20      130
Restaurant20To50          189
toCoupon_GEQ5min            0
toCoupon_GEQ15min           0
toCoupon_GEQ25min           0
direction_same              0
direction_opp               0
Y                           0
dtype: int64

In [10]:
dataset[["car","Bar","CoffeeHouse","CarryAway","RestaurantLessThan20","Restaurant20To50"]].describe(include="all")

,car,Bar,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50
count,108,12577,12467,12533,12554,12495
unique,5,5,5,5,5,5
top,Scooter and motorcycle,never,less1,1~3,1~3,less1
freq,22,5197,3385,4672,5376,6077


In [16]:
dataset['car'].value_counts()

Scooter and motorcycle                      22
Mazda5                                      22
do not drive                                22
crossover                                   21
Car that is too old to install Onstar :D    21
Name: car, dtype: int64

In [17]:
# filling with most common class
df_clean = dataset.apply(lambda x: x.fillna(x.value_counts().index[0]))

In [18]:
df_clean.isnull().sum()

destination             0
passanger               0
weather                 0
time                    0
coupon                  0
expiration              0
gender                  0
maritalStatus           0
has_children            0
income                  0
car                     0
Bar                     0
CoffeeHouse             0
CarryAway               0
RestaurantLessThan20    0
Restaurant20To50        0
toCoupon_GEQ5min        0
toCoupon_GEQ15min       0
toCoupon_GEQ25min       0
direction_same          0
direction_opp           0
Y                       0
dtype: int64

In [19]:
dataset = df_clean
dataset

,destination,passanger,weather,time,coupon,expiration,gender,maritalStatus,has_children,income,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,2PM,Restaurant(<20),1d,Female,Unmarried partner,1,$37500 - $49999,...,never,1~3,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,10AM,Coffee House,2h,Female,Unmarried partner,1,$37500 - $49999,...,never,1~3,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,10AM,Carry out & Take away,2h,Female,Unmarried partner,1,$37500 - $49999,...,never,1~3,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,2PM,Coffee House,2h,Female,Unmarried partner,1,$37500 - $49999,...,never,1~3,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,2PM,Coffee House,1d,Female,Unmarried partner,1,$37500 - $49999,...,never,1~3,4~8,1~3,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12679,Home,Partner,Rainy,6PM,Carry out & Take away,1d,Male,Single,0,$75000 - $87499,...,never,1~3,4~8,1~3,1,0,0,1,0,1
12680,Work,Alone,Rainy,7AM,Carry out & Take away,1d,Male,Single,0,$75000 - $87499,...,never,1~3,4~8,1~3,1,0,0,0,1,1
12681,Work,Alone,Snowy,7AM,Coffee House,1d,Male,Single,0,$75000 - $87499,...,never,1~3,4~8,1~3,1,0,0,1,0,0
12682,Work,Alone,Snowy,7AM,Bar,1d,Male,Single,0,$75000 - $87499,...,never,1~3,4~8,1~3,1,1,1,0,1,0


In [20]:
columns = ['destination',
          'passanger',
          'weather',
          'time',
          'coupon',
          'expiration',
          'gender',
          'has_children',
          'income',
          'Bar',
          'CoffeeHouse',
          'CarryAway',
          'RestaurantLessThan20',
          'Restaurant20To50',
          'toCoupon_GEQ5min',
          'toCoupon_GEQ15min',
          'toCoupon_GEQ25min',
          'direction_same',
          'direction_opp',
          'Y']
not_used_columns = list(set(dataset.columns.to_list()) - set(columns))
dataset = pd.get_dummies(dataset, columns=columns)
dataset.drop(labels=not_used_columns, axis=1, inplace=True)

In [21]:
dataset

,destination_Home,destination_No Urgent Place,destination_Work,passanger_Alone,passanger_Friend(s),passanger_Kid(s),passanger_Partner,weather_Rainy,weather_Snowy,weather_Sunny,...,toCoupon_GEQ15min_0,toCoupon_GEQ15min_1,toCoupon_GEQ25min_0,toCoupon_GEQ25min_1,direction_same_0,direction_same_1,direction_opp_0,direction_opp_1,Y_0,Y_1
0,0,1,0,1,0,0,0,0,0,1,...,1,0,1,0,1,0,0,1,0,1
1,0,1,0,0,1,0,0,0,0,1,...,1,0,1,0,1,0,0,1,1,0
2,0,1,0,0,1,0,0,0,0,1,...,0,1,1,0,1,0,0,1,0,1
3,0,1,0,0,1,0,0,0,0,1,...,0,1,1,0,1,0,0,1,1,0
4,0,1,0,0,1,0,0,0,0,1,...,0,1,1,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12679,1,0,0,0,0,0,1,1,0,0,...,1,0,1,0,0,1,1,0,0,1
12680,0,0,1,1,0,0,0,1,0,0,...,1,0,1,0,1,0,0,1,0,1
12681,0,0,1,1,0,0,0,0,1,0,...,1,0,1,0,0,1,1,0,1,0
12682,0,0,1,1,0,0,0,0,1,0,...,0,1,0,1,1,0,0,1,1,0


In [22]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [46]:
#Apriori min support
min_support = 0.5
#Max lenght of apriori n-grams
max_len = 5
frequent_items = apriori(dataset, use_colnames=True, min_support=min_support, max_len=max_len + 1)
rules = association_rules(frequent_items, metric='lift', min_threshold=1)
rules.head(10).sort_values(by='confidence', ascending=False)

C:\Users\Lenovo\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(passanger_Alone),(toCoupon_GEQ5min_1),0.575922,1.000000,0.575922,1.000000,1.000000,0.000000,inf
2,(weather_Sunny),(toCoupon_GEQ5min_1),0.793835,1.000000,0.793835,1.000000,1.000000,0.000000,inf
6,(expiration_1d),(toCoupon_GEQ5min_1),0.559051,1.000000,0.559051,1.000000,1.000000,0.000000,inf
9,(gender_Female),(toCoupon_GEQ5min_1),0.513324,1.000000,0.513324,1.000000,1.000000,0.000000,inf
4,(weather_Sunny),(toCoupon_GEQ25min_0),0.793835,0.880874,0.729423,0.918860,1.043123,0.030155,1.468157
5,(toCoupon_GEQ25min_0),(weather_Sunny),0.880874,0.793835,0.729423,0.828068,1.043123,0.030155,1.199107
3,(toCoupon_GEQ5min_1),(weather_Sunny),1.000000,0.793835,0.793835,0.793835,1.000000,0.000000,1.000000
1,(toCoupon_GEQ5min_1),(passanger_Alone),1.000000,0.575922,0.575922,0.575922,1.000000,0.000000,1.000000
7,(toCoupon_GEQ5min_1),(expiration_1d),1.000000,0.559051,0.559051,0.559051,1.000000,0.000000,1.000000
8,(toCoupon_GEQ5min_1),(gender_Female),1.000000,0.513324,0.513324,0.513324,1.000000,0.000000,1.000000


In [47]:
len(rules)

184

In [25]:
dataset['Y_0'].value_counts()

0    7210
1    5474
Name: Y_0, dtype: int64

In [26]:
dataset['Y_1'].value_counts()

1    7210
0    5474
Name: Y_1, dtype: int64

In [56]:
#Apriori min support
min_support = 0.25
#Max lenght of apriori n-grams
max_len = 2
frequent_items = apriori(dataset, use_colnames=True, min_support=min_support, max_len=max_len + 1)
rules = association_rules(frequent_items, metric='lift', min_threshold=1)
target = '{\'Y_1\'}'
results_coupon_accpeted = rules[rules['consequents'].astype(str).str.contains(target, na=False)].sort_values(by='confidence', ascending=False)
results_coupon_accpeted.head(10)

C:\Users\Lenovo\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
503,"(weather_Sunny, expiration_1d)",(Y_1),0.432356,0.568433,0.282245,0.652808,1.148435,0.036480,1.243023
246,"(weather_Sunny, destination_No Urgent Place)",(Y_1),0.413277,0.568433,0.267739,0.647844,1.139703,0.032819,1.225502
806,"(expiration_1d, toCoupon_GEQ25min_0)",(Y_1),0.487149,0.568433,0.312520,0.641528,1.128591,0.035608,1.203907
374,"(destination_No Urgent Place, direction_opp_1)",(Y_1),0.495348,0.568433,0.313939,0.633774,1.114949,0.032367,1.178417
367,"(direction_same_0, destination_No Urgent Place)",(Y_1),0.495348,0.568433,0.313939,0.633774,1.114949,0.032367,1.178417
320,"(destination_No Urgent Place, toCoupon_GEQ5min_1)",(Y_1),0.495348,0.568433,0.313939,0.633774,1.114949,0.032367,1.178417
17,(destination_No Urgent Place),(Y_1),0.495348,0.568433,0.313939,0.633774,1.114949,0.032367,1.178417
356,"(destination_No Urgent Place, toCoupon_GEQ25mi...",(Y_1),0.495348,0.568433,0.313939,0.633774,1.114949,0.032367,1.178417
83,(expiration_1d),(Y_1),0.559051,0.568433,0.349732,0.625582,1.100538,0.031949,1.152634
782,"(expiration_1d, toCoupon_GEQ5min_1)",(Y_1),0.559051,0.568433,0.349732,0.625582,1.100538,0.031949,1.152634


In [57]:
len(results_coupon_accpeted)

36